In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from libtiff import TIFF
import cv2

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

In [2]:
import os
file_lst = []
os.chdir("/Users/minruilu/Desktop/OneDrive - UNSW/UNSW/2020 Semester 2/comp9517/project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/Sequence 1")
for root, dirs, files in os.walk(".", topdown=False):
    for name in files:
        if name.endswith("tif"):
            file_lst.append(os.path.join(root, name))
if not os.path.exists("THRESHOLD"):
    os.mkdir("THRESHOLD")

In [102]:
# For dataset 2 Fluo-N2DL-HeLa
def watershed_batch_process_2(img_path): 
    tif = TIFF.open(img_path,mode='r')
    image = tif.read_image()
    sobelX = cv2.Sobel(image,cv2.CV_64F,1,0)# gradient in x direction
    sobelY = cv2.Sobel(image,cv2.CV_64F,0,1)# gradient in y direction
    sobelX = np.uint8(np.absolute(sobelX))
    sobelY = np.uint8(np.absolute(sobelY))
    image = cv2.bitwise_or(sobelX,sobelY)
    ret, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.medianBlur(image, 9)
    image = cv2.dilate(image, np.ones((3,3), np.uint8))
    image = cv2.erode(image, np.ones((19,19), np.uint8))
    image = cv2.dilate(image, np.ones((13,13), np.uint8))
    distance = ndi.distance_transform_edt(image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((9, 9)),
                                labels=image)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=image)
    labels = cv2.normalize(labels,None, 0, 255, cv2.NORM_MINMAX)
    labels[np.where(labels>10)] = 255
    cv2.imwrite("./WATERSHED/"+img_path,labels)

In [103]:
# For dataset 3 PhC-C2DL-PSC
def watershed_batch_process_3(img_path): 
    tif = TIFF.open(img_path,mode='r')
    image = tif.read_image()
    sobelX = cv2.Sobel(image,cv2.CV_64F,1,0)# the gradient of x direction
    sobelY = cv2.Sobel(image,cv2.CV_64F,0,1)# the gradient of y direction
    sobelX = np.uint8(np.absolute(sobelX))
    sobelY = np.uint8(np.absolute(sobelY))
    image = cv2.bitwise_or(sobelX,sobelY)#
    ret, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.medianBlur(image, 5)
    image = cv2.dilate(image, np.ones((3,3), np.uint8))
    image = cv2.erode(image, np.ones((11,11), np.uint8))
    image = cv2.dilate(image, np.ones((7,7), np.uint8))
    distance = ndi.distance_transform_edt(image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((13, 13)),
                            labels=image)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=image)
    labels = cv2.normalize(labels,None, 0, 255, cv2.NORM_MINMAX)
    labels[np.where(labels>10)] = 255
    cv2.imwrite("./WATERSHED/"+img_path,labels)

In [ ]:
# For dataset 3 PhC-C2DL-PSC
def shreshold_batch_process_3(img_path):

In [104]:
for img_path in file_lst:
    watershed_batch_process_3(img_path)